# Abstractive Summarization

This notebook contains a sample for abstractive summarization using chain of density prompting.

In [1]:
!pip install python-dotenv openai

In [2]:
from dotenv import load_dotenv
import logging
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.INFO)

load_dotenv()

True

In [3]:
from biagen.llm import CohereProvider, GroqProvider, OpenAIProvider

#llm = CohereProvider.from_env()
#llm = GroqProvider.from_env()
llm = OpenAIProvider.from_url("http://127.0.0.1:5000/v1",
                              api_key="e18cbf602915b707311f9601c178e23c",
                              model_name="CohereForAI_c4ai-command-r-08-2024-exl2")

In [4]:
from typing import List

logger = logging.getLogger('abstractive_summary')

def abstractive_summary(article: str, iterations: int = 3) -> List[str]:
    summaries = []
    
    header = f"""You are a copy assistant at an outdoorsman magazine, with a master's degree in journalism. Your task is to provide analysis of the following article with precision.
    
## Article

```article
{article}
```
"""
    
    ## Analysis
    
    prompt_a = f"""{header}
We first must perform an analysis on the article, to identify the most important information.

## Analysis (the most significant topics, with one sentence commentary)

"""

    analysis = llm.generate_one(prompt_a, max_tokens=1024, temperature=0.7, stop_sequences=None)
    alen = len(analysis.split())

    logger.debug(f"analysis {alen} {analysis}")

    # Summarize

    prompt_s = f"""{header}
Given the article and the following analysis, provide a detailed, erudite, succinct, and accurate summary.

## Analysis

{analysis}

## Summary (here is a 5 paragraph summary)

"""

    summary = llm.generate_one(prompt_s, max_tokens=1024, temperature=0.5, stop_sequences=None)

    summaries.append(summary)
    slen = len(summary.split())
    logger.debug(f"summary {slen} {summary}")

    ### Improvement Loop

    for i in range(iterations):
        # Get our last summary

        summary = summaries[-1]

        # Check for missing information in the summary

        prompt_m = f"""{header}
## Current Summary

{summary}

Given the article and the current summary, identify missing information or ways to improve the summary.

## Missing Information (here are 12 entries of novel information not contained in the summary)

"""

        missing = llm.generate_one(prompt_m, max_tokens=1024, temperature=0.7, stop_sequences=None)
        mlen = len(missing.split())
        logger.debug(f"missing {mlen} {missing}")

        # Check for most important information in the summary

        prompt_r = f"""{header}
## Current Summary

{summary}

## Missing Information

{missing}

Using the current summary and the identified missing information, determine if our summary is six (6) paragraphs, identify what information is important and unimportant.

We should be sure to include important facts like size, phone number, unique features, wildlife, history, and contact information.

## Most Important and Interesting Information (here are 12 entries, with one sentence commentary)

"""

        important = llm.generate_one(prompt_r, max_tokens=2048, temperature=0.5, stop_sequences=None)
        ilen = len(important.split())
        logger.debug(f"important {ilen} {important}")

        # Resummarize

        prompt_r = f"""{header}
## Current Summary

{summary}

## Missing Information

{missing}

## Ideas To Consider

{important}

Do not mention the Wildlife Code.

Contact information should be included.

Cover overview, location, features, history, wildlife, game, camping, recreation, contact and management

Do not be brief.

Should be six (6) paragraphs. 

Using the current summary and the identified missing information, here is an improved summary of the article.

## Improved Summary (here is a reorganized and expanded version, including missing information and ideas; six (6) detailed paragraphs)

"""

        resummarized = llm.generate_one(prompt_r, max_tokens=2048, temperature=0.5, stop_sequences=None)
        rlen = len(resummarized.split())
        logger.debug(f"resummarized {rlen} {resummarized}")
        #print(prompt_r, resummarized, rlen)

        summaries.append(resummarized)

    return summaries


In [5]:
# load some data
results_file = 'areas_results.02.jsonl'
details_file = 'areas_details.02.jsonl'

scraped_lakes = pd.read_csv("random_100_areas.tsv", sep='\t')


In [6]:
import os

from biagen.io import append_to_jsonl, read_jsonl

results = read_jsonl(results_file) if os.path.exists(results_file) else []
processed = set(r['area_id'] for r in results)
len(results)

0

In [7]:

model_desc = f"{llm.model}"

for i, row in scraped_lakes.iterrows():
    category = row['category']
    subcategory = row['subcategory']
    area_id = row['area_id']
    area_name = row['area_name']
    article = row['area_info']

    if area_id in processed:
        continue

    summaries = abstractive_summary(article)

    append_to_jsonl({
        'area_id': area_id,
        'area_name': area_name,
        'category': category,
        'subcategory': subcategory,
        'summary': summaries[-1],
        'model': llm.model,
    }, results_file)

    append_to_jsonl({
        'area_id': area_id,
        'area_name': area_name,
        'category': category,
        'subcatgory': subcategory,
        'summary': summaries[-1],
        'summaries': summaries,
        'model': llm.model,
    }, details_file)

    print('=' * 50)
    print(f"{area_name} ({area_id})")
    print('=' * 50)
    for i, s in enumerate(summaries):
        print(f"Round {i}: {s}")
        print('=' * 50)
    print()




INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"


Bolivar Forestry Office (bolivar-forestry-office)
Round 0: Nestled within the city limits of Bolivar, the Bolivar Forestry Office serves as a hub for field staff operating in Bolivar and its surrounding counties. This office is open to the public from Monday to Friday, offering a range of services and information. The area's comprehensive set of rules and regulations govern various outdoor activities, ensuring a safe and enjoyable experience for visitors. Special regulations prohibit certain activities, such as target shooting and field trials, while others require special use permits, including equestrian use, camping for large groups, and trapping. Vehicle use is restricted to designated roads and parking areas, with a speed limit of 45 miles per hour. Bicycles and horses are also subject to specific rules, with designated trails and roads for their use. Camping is permitted only in designated areas, with strict adherence to quiet hours, campsite limits, and stay duration. From Septe

INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"


Branson Forestry Office (branson-forestry-office)
Round 0: The Branson Forestry Office, located in Taney and Stone counties, serves as a vital hub for field staff and outdoor enthusiasts. This office provides a comprehensive base of operations, offering essential services and resources for those exploring the region's natural wonders. The article delves into the regulations governing the surrounding conservation area, emphasizing the need for adherence to rules regarding camping, target shooting, vehicle use, and the protection of wildlife and vegetation. It highlights the significance of consulting the Wildlife Code and obtaining special permits for various activities, ensuring a harmonious balance between human enjoyment and ecological preservation. 

Furthermore, the article outlines the accessibility and hours of operation for the area. Visitors can enjoy the outdoors during extended hours from 4:00 AM to 10:00 PM every day of the week. Clear directions are provided for reaching th

INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"


...

Kessler Memorial Wildlife Area (kessler-memorial-wildlife-area)
Round 0: The Kessler Memorial Wildlife Area, situated in Lincoln County, Missouri, spans 157 acres and is dedicated to preserving and enhancing wildlife habitats. Donated by David L. Kessler in 1971, the area is managed by the Conservation Commission, with a focus on providing food and shelter for various species. Accessible year-round, the wildlife area boasts diverse habitats, including 64 acres of woods and 93 acres of old fields and grasslands. 

Regulations are in place to ensure the protection of both the wildlife and the natural environment. These rules cover a range of activities, from camping and target shooting to the use of vehicles and horses, and the collection of wild plants and mushrooms. Special use permits are required for certain activities, emphasizing the careful management of this natural space. 

Hunting and fishing are permitted within specific regulations and seasons. Deer and turkey hunting, for in

INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"


Linscomb Wildlife Area (linscomb-wildlife-area)
Round 0: Nestled in Missouri, the Linscomb Wildlife Area, spanning 2,384 acres, offers a diverse natural haven for outdoor enthusiasts. Formerly known as Winding River Ranch, this area is managed by the Missouri Department of Conservation and features a range of habitats, including reconstructed prairies, oak and hickory savannas, and mixed grass fields. The wildlife area is accessible to the public daily, with specific hours from 4:00 a.m. to 10:00 p.m., and various regulations govern activities to protect the environment and ensure visitor safety. Bird watching, fishing for catfish, and hunting for deer, turkey, rabbit, and squirrel are among the popular outdoor pursuits available. Additionally, the area boasts unique natural features such as small ponds, the Osage River Arm of Truman Reservoir, and Simms Creek, providing a captivating setting for exploration. Furthermore, the article provides detailed information on the fall deer and t

INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"


: 